# **Object Detection**

<a target="_blank" href="https://github.com/amaciag/CV-Experiments/blob/main/Shampoo_Object_Detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"/>View source on GitHub</a>

Object Detection is the supervised learning process of locating and identifying objects in an image or video. This tutorial shows how to use some Google tensorflow pretrained model to train on a shampoo image dataset. Transfer learning and data augmentation techniques are incorporated.

## **Install and import appropriate modules and their dependencies**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import io
import json
import os
import pandas as pd
import requests
import tensorflow as tf

from google.colab import drive
from IPython.display import Javascript
from PIL import Image

def resize_output_cell():
  display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

get_ipython().events.register('pre_run_cell', resize_output_cell)
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
# Replace setup.py in models repo with the following code

%%writefile models/research/object_detection/packages/tf2/setup.py
"""Setup script for object_detection with TF2.0."""
import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    # Required for apache-beam with PY3
    'avro-python3',
    'apache-beam',
    'pillow',
    'lxml',
    'matplotlib',
    'Cython',
    'contextlib2',
    'tf-slim',
    'six',
    'pycocotools',
    'lvis',
    'scipy',
    'pandas',
    'tf-models-official==2.7.0',
    'tensorflow_io',
    'keras'
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)



In [ ]:
# Install object_detection package and its dependecies

%%bash

sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
pip install opencv-python-headless==4.1.2.30

## **Create reuseable functions**

In [ ]:
from object_detection.utils import dataset_util, label_map_util

def create_tf_example(data):
    
    # print(data['Labeled Data'])
    response = requests.get(data['Labeled Data'])
    encoded_jpg = response.content
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for obj in data['Label']['objects']:
        bbox = obj['bbox']
        xmin = bbox['left']
        ymin = bbox['top']
        xmax = xmin + bbox['width']
        ymax = ymin + bbox['height']
        xmins.append(xmin / width)
        xmaxs.append(xmax / width)
        ymins.append(ymin / height)
        ymaxs.append(ymax / height)
        classes_text.append(obj['title'].encode('utf8')) 
        classes.append(label_map_dict[obj['title']])
    
    filename = data['External ID'].encode('utf8')
    image_format = b'jpg'
    tf_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/width': dataset_util.int64_feature(width),
            'image/filename': dataset_util.bytes_feature(filename),
            'image/source_id': dataset_util.bytes_feature(filename),
            'image/encoded': dataset_util.bytes_feature(encoded_jpg),
            'image/format': dataset_util.bytes_feature(image_format),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
    return tf_example

def generate_tf_record(json_input, output_path, label_map):

    global label_map_dict
    label_map_dict = label_map_util.get_label_map_dict(label_map)
    
    with tf.io.gfile.GFile(json_input, 'r') as file:
        dataset = json.load(file)
    
    writer = tf.io.TFRecordWriter(output_path)
    for data in dataset:
        tf_example = create_tf_example(data)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print(f'Successfully created TF-records {output_path}')

def update_pipeline_config(pipeline_config, args):
  
  pipeline_config['model'].ssd.num_classes=args['num_classes']
  pipeline_config['train_config'].batch_size=args['batch_size']
  pipeline_config['train_config'].fine_tune_checkpoint_type=args['fine_tune_checkpoint_type']
  pipeline_config['train_config'].fine_tune_checkpoint=args['fine_tune_checkpoint_path']
  pipeline_config['eval_input_config'].label_map_path=args['label_map_path']
  pipeline_config['eval_input_config'].tf_record_input_reader.input_path[0]=args['eval_input_path']
  pipeline_config['train_input_config'].label_map_path=args['label_map_path']
  pipeline_config['train_input_config'].tf_record_input_reader.input_path[0]=args['train_input_path']
  
  return pipeline_config

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

def get_prediction(image_url):

  response = requests.get(image_url)
  encoded_jpg = response.content
  encoded = io.BytesIO(encoded_jpg)
  image_np = load_image_into_numpy_array(encoded)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.  
  num_detections = int(detections.pop('num_detections'))
  detections = {
      key: value[0, :num_detections].numpy()
      for key, value in detections.items()
  }
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  return image_np, detections


## **Create directories and download shampoo image data**

In [ ]:
%%bash

if [ ! -d workspace ]; then
  mkdir -p workspace/{pretrained_models,trained_models,data}
fi

wget -O workspace/data/eval.json https://raw.githubusercontent.com/amaciag/CV-Experiments/main/data/eval.json
wget -O workspace/data/train.json https://raw.githubusercontent.com/amaciag/CV-Experiments/main/data/train.json

In [ ]:
items = [{
  'id': '1',
  'name': '"Loreal"'
},
{
  'id': '2',
  'name': '"Pantene"'
},
{
  'id': '3',
  'name': '"Head_&_Shoulders"'
}]

expression = ''
for item in items:
  expression += 'item {\n'
  for key,value in item.items():
    expression += 2*' ' + key + ': ' + value + '\n'
  expression += '}\n'

with open('workspace/data/shampoo_label_map.pbtxt', 'w') as f:
  f.write(expression)
print(expression)

## **Save image data as tf records**

In [ ]:
generate_tf_record('workspace/data/train.json', 'workspace/data/train.record', 'workspace/data/shampoo_label_map.pbtxt')
generate_tf_record('workspace/data/eval.json', 'workspace/data/eval.record', 'workspace/data/shampoo_label_map.pbtxt')

# !python generate_tf_record.py --json-input workspace/data/train.json --output-path workspace/data/train.record --label-map workspace/data/shampoo_label_map.pbtxt
# !python generate_tf_record.py --json-input workspace/data/eval.json --output-path workspace/data/eval.record --label-map workspace/data/shampoo_label_map.pbtxt

In [ ]:
MODELS = {
    "SSD MobileNet v2 320x320":"http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz",
    "EfficientDet D0 512x512": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz",
    "EfficientDet D2 768x768": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz"
}

selected_model = "SSD MobileNet v2 320x320"
file = MODELS[selected_model].split('/')[-1]

In [ ]:
!wget {MODELS[selected_model]} --directory-prefix workspace/pretrained_models
!tar -xf workspace/pretrained_models/{file} -C workspace/pretrained_models

In [ ]:
from object_detection.utils import config_util

model_dir = file.replace('.tar.gz', '')
pipeline_config_path = f'workspace/pretrained_models/{model_dir}/pipeline.config'
pipeline_config = config_util.get_configs_from_pipeline_file(pipeline_config_path)

args = {'num_classes': 3, 
        'batch_size': 4, 
        'fine_tune_checkpoint_type': 'detection',
        'fine_tune_checkpoint_path': f'workspace/pretrained_models/{model_dir}/checkpoint/ckpt-0',
        'label_map_path': 'workspace/data/shampoo_label_map.pbtxt',
        'eval_input_path': 'workspace/data/eval.record',
        'train_input_path': 'workspace/data/train.record'}

pipeline_config = update_pipeline_config(pipeline_config, args)        
pipeline_config = config_util.create_pipeline_proto_from_configs(pipeline_config)
config_util.save_pipeline_config(pipeline_config, 'workspace/trained_models')

## **Train a pretrained model**

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=drive/MyDrive/CV-Experiments-main/ssd_mobilenet/v1

In [ ]:
!python models/research/object_detection/model_main_tf2.py --pipeline_config_path=workspace/trained_models/pipeline.config \
  --model_dir=drive/MyDrive/CV-Experiments-main/ssd_mobilenet/v1 --checkpoint_every_n=100 --num_train_steps=1000 --alsologtostderr

In [ ]:
# LOG_DIR = 'workspace/efficientdet_d0/v1'
# get_ipython().system_raw(
#     'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#     .format(LOG_DIR)
# )

In [ ]:
# get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
# !curl -s http://localhost:4040/api/tunnels | python -c \
#   "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## **Get and visualize predictions**

In [ ]:
# %%bash
!cp models/research/object_detection/exporter_main_v2.py .
!python exporter_main_v2.py --input_type image_tensor \
  --pipeline_config_path workspace/trained_models/pipeline.config  \
  --trained_checkpoint_dir drive/MyDrive/CV-Experiments-main/ssd_mobilenet/v1 \
  --output_directory sample_model

In [ ]:
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = './sample_model/saved_model'

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

In [ ]:
import json
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

with tf.io.gfile.GFile('workspace/data/eval.json', 'r') as file:
  eval_data = json.load(file)

category_index = label_map_util.create_category_index_from_labelmap('./workspace/data/shampoo_label_map.pbtxt', use_display_name=True)

for data in eval_data:
    
    url = data['Labeled Data']
    image_np, detections = get_prediction(url)
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    
    shampoo = category_index[detections['detection_classes'][np.argmax(np.max(detections['detection_multiclass_scores'], axis=1))]]['name']
    plt.figure(figsize=(20,20))
    plt.title(shampoo)
    plt.imshow(image_np)
    # plt.cla()
    # print('Done')
plt.show()